In [13]:
# 01_market_etl.ipynb

import os
import pandas as pd
import yfinance as yf
from datetime import date, timedelta

# --- Settings
TICKERS = ["NVDA", "MSFT", "AAPL", "META", "AVGO", "AMD", "TSM", "ASML"]
END = date.today()
START = END - timedelta(days=730)  # ~2 years so you can compute MA200 etc.

# --- Download with 'group_by=ticker' to simplify column handling
raw = yf.download(
    TICKERS,
    start=str(START),
    end=str(END),
    auto_adjust=False,     # keep both Close and Adj Close if available
    group_by="ticker",     # top-level columns become ticker symbols
    progress=False
)

# --- Build a tidy dataframe: Date | Ticker | Price
frames = []
for t in TICKERS:
    if t not in raw.columns.get_level_values(0):
        print(f"⚠️  Skipping {t}: not returned by yfinance")
        continue
    sub = raw[t].copy()
    # Prefer 'Adj Close'; fall back to 'Close'
    price_col = "Adj Close" if "Adj Close" in sub.columns else ("Close" if "Close" in sub.columns else None)
    if price_col is None:
        print(f"⚠️  Skipping {t}: no 'Adj Close' or 'Close' found")
        continue

    sub = sub.reset_index()[["Date", price_col]].dropna()
    sub.rename(columns={price_col: "Price"}, inplace=True)
    sub["Ticker"] = t
    frames.append(sub)

if not frames:
    raise RuntimeError("No data fetched for any ticker. Check tickers or date range.")

df = pd.concat(frames, ignore_index=True)

# --- Add returns & simple indicators
df.sort_values(["Ticker", "Date"], inplace=True)
df["Daily_Return"] = df.groupby("Ticker")["Price"].pct_change()

def add_indicators(g):
    g = g.copy()
    g["MA20"]  = g["Price"].rolling(20, min_periods=1).mean()
    g["MA50"]  = g["Price"].rolling(50, min_periods=1).mean()
    g["MA200"] = g["Price"].rolling(200, min_periods=1).mean()
    # Annualised volatility (20-day window on daily returns)
    g["Vol20"] = g["Daily_Return"].rolling(20).std() * (252**0.5)
    return g

df = df.groupby("Ticker", group_keys=False).apply(add_indicators)
df = df.dropna(subset=["Daily_Return"]).reset_index(drop=True)

# --- Save to CSV
os.makedirs("../data", exist_ok=True)
csv_path = "../data/ai_stocks.csv"
df.to_csv(csv_path, index=False)

print(f"✅ Saved {len(df):,} rows to {csv_path}")
df.head()


✅ Saved 3,984 rows to ../data/ai_stocks.csv


/tmp/ipykernel_12254/3941308084.py:59: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Ticker", group_keys=False).apply(add_indicators)


Price,Date,Price,Ticker,Daily_Return,MA20,MA50,MA200,Vol20
0,2023-09-26,170.291992,AAPL,-0.023399,172.332016,172.332016,172.332016,NaN
1,2023-09-27,168.776840,AAPL,-0.008897,171.146957,171.146957,171.146957,NaN
2,2023-09-28,169.034317,AAPL,0.001526,170.618797,170.618797,170.618797,NaN
3,2023-09-29,169.549271,AAPL,0.003046,170.404892,170.404892,170.404892,NaN
4,2023-10-02,172.064636,AAPL,0.014836,170.681516,170.681516,170.681516,NaN
